In [ ]:
!pip install matplotlib-venn

In [ ]:
!apt-get -qq install -y libfluidsynth1

Selecting previously unselected package libfluidsynth1:amd64.
(Reading database ... 160815 files and directories currently installed.)
Preparing to unpack .../libfluidsynth1_1.1.9-1_amd64.deb ...
Unpacking libfluidsynth1:amd64 (1.1.9-1) ...
Setting up libfluidsynth1:amd64 (1.1.9-1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.2) ...
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link



#합성곱 신경망 소개
컨브넷 정의와 컨브넷이 컴퓨터 비전 관련 작업에 잘 맞는 이유에 대해 이론적 배경을 알아봅시다. 하지만 먼저 간단한 컨브넷 예제를 둘러보져. 2장에서 완전 연결 네트워크로 풀었던 MNIST 숫자 이미지 분류에 컨브넷을 사용해 보겠습니다. 기본적인 컨브넷이더라도 2장에서 다룬 완전 연결된 모델의 성능을 훨씬 앞지를 것입니다.

다음 코드는 기본적인 컨브넷의 모습입니다. Conv2D와 MaxPooling2D 층을 쌓아 올렸습니다. 잠시 후에 이들이 무엇인지 배웁니다.

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation = 'relu', input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation ='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))

컨브넷이(image_height, image_width, image_channels)크기의 입력 텐서를 사용한다는 점이 중요합니다(배치 차원은 포함하지 않습니다). 이 예제에서는 MNIST이미지 포맷인 (28,28,1)크기의 입력을 처리하도록 컨브넷을 설정해야 합니다. 이 때문에 첫 번째 층의 매개변수로 input_shape=(28,28,1)을 전달했습니다.

지금까지의 컨브넷 구조를 출력해 보죠

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Conv2D와 MaxPooling2D층의 출력은 (height, width, channels)크기의 3D 텐서입니다. 높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있습니다. 채널의 수는 Conv2D층에 전달된 첫 번쨰 매개변수에 의해 조절됩니다(32개 또는 64개).

다음 단계에서의 마지막 층의((3,3,64) 크기인)출력 텐서를 완전 연결 네트워크에 주입합니다. 이 네트워크는 이미 익숙하게 보았던 Dense층을 쌓은 분류기입니다 이 분류기는 1D벡터를 처리하는데, 이전 층의 출력이 3D 텐서입니다. 그래서 먼저 3D출력을 1D텐서로 펼쳐야 합니다. 그다음 몇 개의 Dense층을 추가합니다.


In [ ]:
#컨브넷 위에 분류기 추가하기
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

여기에서 볼 수 있듯이 (3, 3, 64)출력이 (576,) 크기의 벡터로 펼쳐진 후 Dense 층으로 주입되었습니다.

이제 MNIST숫자 이미지에 이 컨브넷을 훈련합니다. 2장의MNIST예제 코드를 많이 재사용하겠습니다.

책이랑 코드가 좀 달라졌습니다. 이점 유의해 주시기 바랍니다.
(https://stackoverflow.com/questions/67703871/importerror-cannot-import-name-to-categorical-from-keras-utils-usr-local)

In [ ]:
from keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
model.fit(train_images, train_labels, epochs = 5, batch_size=64)

11501568/11490434 [==============================] - 0s 0us/step
Epoch 1/5
938/938 [==============================] - 70s 58ms/step - loss: 0.3970 - accuracy: 0.8726
Epoch 2/5
938/938 [==============================] - 56s 59ms/step - loss: 0.0500 - accuracy: 0.9844
Epoch 3/5
938/938 [==============================] - 54s 58ms/step - loss: 0.0318 - accuracy: 0.9904
Epoch 4/5
938/938 [==============================] - 54s 58ms/step - loss: 0.0238 - accuracy: 0.9923
Epoch 5/5
938/938 [==============================] - 54s 58ms/step - loss: 0.0196 - accuracy: 0.9941


테스트 데이터에서 모델을 평가해 보죠

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 4s 9ms/step - loss: 0.0245 - accuracy: 0.9921


0.9921000003814697

2장의 완전 연결 네트워크는 97.8%의 테스트 정확도를 얻은 반면에 기본적인 컨브넷의 99.2%의 테스트 정확도를 얻었습니다. 에려율이 (상대적으로)64%나 줄었습니다. 나쁘지 않군요!

완전 연결된 모델보다 왜 간단한 컨브넷이 더 잘 작동할까요? 이에 대해 알아보기 위해 Conv2D와 MaxPooling2D층이 어떤 일을 하는지 살펴보겠습니다.

## 합성곱 연산

완전 연결층과 합성곱 층 사이의 근본적인 차이는 다음과 같습니다. Dense층은 입력 특성 공간에 있는 전역 패턴(예를 들어 MNIST 숫자 이미지에서는 모든 픽셀에 걸친 패턴)을 학습하지만 합성곱 층은 지역 패턴을 학습합니다. 이미지일 경우 작은 2D윈도우로 입력에서 패턴을 찾습니다. 앞의 예에서 이 윈도우는 모두 3*3크기였습니다.


이 핵심 특징은 컨브넷에 두 가지 흥미로운 성질을 제공합니다.

 * 학습된 패턴은 평행 이동 불변성을 가집니다. 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳(예를들어 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있습니다. 완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야 합니다. 이런 성질은 컨브넷이 이미지를 효율적으로 처리하게 만들어 줍니다.(근본적으로 우리가 보는 세상을 평행 이동으로 인해 다르게 인식되지 않습니다). 적은 수의 훈련 샘플을 사용해서 일반화 능력을 가진 표현을 학습할 수 있습니다.

 * 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있습니다 첫 번째 합성곱 층이 예지 같은 작은 지역 패턴을 학습합니다. 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식입니다. 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있습니다(근본적으로 우리가 보는 세상은 공간적 계층 구조를 가집니다).

 합성곱 여난은 특성 맵이라고 부르는 3D텐서에 적용됩니다, 이 텐서는 2개의 공간축과 깊이축으로 구성됩니다. RGB이미지는 3개의 컬러체널(빨간색, 녹색, 파란색)을 가지므로 깊이 축의 차원이 3이 됩니다. MNIST숫자처럼 흑백 이미지는 깊이 축의 차원이 1(회색톤)입니다. 합성곱 연산은 입력 특성 맵에서 작은 패치들을 추출하고 이런 모든 패치에 같은 변환을 적용하여 출력특성맵을 만듭니다.

 출력 특성 맵도 높이와 너비를 가진 3D텐서입니다. 출력 텐서의 깊이는 층의 매개변수로 결정되기 때문에 상황에 따라 다릅니다. 이렇게 되면 깊이 축의 채널은 더 이상 RGB입력처럼 특정 컬러를 의미하지 않습니다. 그 대신 일종의 필터를 의미합니다. 필터는 입력 데이터의 어떤 특성을 인쾽합니다. 예를 들어 고수준으로 보면 하나의 필터가 '입력에 얼굴이 있는지'를 인코딩 할 수 있습니다.

 MNIST예제에서는 첫 번째 합성곱 층이 (28,28,1) 크기의 특성 맵을 입력으로 받아 (26,26,32) 크기의 특성 맵을 출력합니다. 즉 입력에 대해 32개의 필터를 적용합니다. 32개의 출력 채늘 각각은 26*26 크기의 배열 값을 가집니다. 이 값은 입력에 대한 필터의 응답 맵 입니다. 입력의 각 위치에서 필터 패턴에 대한 응답을 나타냅니다. 특성 맵이란 말이 의미하는 것은 다음과 같습니다. 깊이 축에 있는 각 차원은 하나의 특성이고, 2D텐서 output[:,:,:n]은 입력에 대한 이 필터 응답을 나타내는 2D공간상의 맵입니다.
 

케라스의 Conv2D 층에서 이 피라미터는 Conv2D(output_depth,(window_height, window_width)) 처럼 첫 번쨰와 두 번째 매개변수로 전달됩니다.

3D 입력 특성 맵 위를 3X3 또는 5X5 크기의 윈도우가 슬라이딩 하면서 모든 위치에서 3D특성패치를 추출하는 방식으로 합성곱이 작동합니다. 이런 3D패치는 (output_depth,)크기의 3D특성맵으로 재구성 되니다. 출력 특성 맵의 공간상 위치는입력 특성 맵의 같은 위치에 대응됩니다.(예를 들어 출력의 오른쪽 아래 모서리는 입력의 오른쪽 아래 부근에 해당하는 정보를 담고 있습니다). 3*3윈도우를 사용하면 3D패치 input[i-1:i+2, j-1:j+2,:]로부터 벡터 output [i,j,:]가 만들어집니다. 책에 전체 과정이 자세히 나타나 있습니다.